In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
files_test = []
train_patch = '/kaggle/input/transformer-time/data_train/data_train/'
test_patch = '/kaggle/input/transformer-time/data_test/data_test/'
df_train = pd.read_csv('/kaggle/input/transformer-time/train.csv')

for _, _, filenames in os.walk('/kaggle/input/transformer-time/data_test'):
    for filename in filenames:
        files_test.append(filename)

In [3]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
import catboost as cb

In [4]:
X = []
y = []

for row in df_train.iterrows():
    df = pd.read_csv(train_patch + row[1]['id'])
    X.append(np.hstack((df.iloc[-1].values, 
           df.iloc[-100].values, df.iloc[-1].values - df.iloc[-100].values, df.iloc[-100:].mean(), df.iloc[-100:].std(), 
           df.iloc[-200].values, df.iloc[-1].values - df.iloc[-200].values, df.iloc[-200:].mean(), df.iloc[-200:].std(), 
           df.iloc[-300].values, df.iloc[-1].values - df.iloc[-300].values, df.iloc[-300:].mean(), df.iloc[-300:].std())))
    y.append(row[1]['predicted'])
X = np.array(X)
y = np.array(y)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=6)

In [6]:
tree_params = {
    'iterations': 10000,
    'learning_rate': 0.1,
    'eval_metric': 'MAE',
    'loss_function': 'RMSE',
    'random_state': 6,
    'depth': 4,
    'l2_leaf_reg': 10,
    'learning_rate': 0.1,
    'border_count': 20,
    'task_type': 'CPU',
    'devices': '0:1'
}

model = cb.CatBoostRegressor(**tree_params)

In [7]:
model.fit(x_train, y_train, eval_set=(x_test, y_test), verbose_eval=1000, use_best_model=True, plot=False)

0:	learn: 207.9936955	test: 216.5054047	best: 216.5054047 (0)	total: 52.8ms	remaining: 8m 47s
1000:	learn: 37.6277424	test: 84.3872168	best: 83.9309333 (671)	total: 1.56s	remaining: 14s
2000:	learn: 20.3292096	test: 85.4271518	best: 83.9309333 (671)	total: 2.98s	remaining: 11.9s
3000:	learn: 11.9329181	test: 85.6753565	best: 83.9309333 (671)	total: 4.44s	remaining: 10.4s
4000:	learn: 7.2113965	test: 86.1157903	best: 83.9309333 (671)	total: 5.95s	remaining: 8.92s
5000:	learn: 4.4220489	test: 86.4843330	best: 83.9309333 (671)	total: 7.39s	remaining: 7.39s
6000:	learn: 2.7591994	test: 86.5780382	best: 83.9309333 (671)	total: 9.14s	remaining: 6.09s
7000:	learn: 1.7714133	test: 86.6931246	best: 83.9309333 (671)	total: 10.6s	remaining: 4.56s
8000:	learn: 1.1404682	test: 86.7237682	best: 83.9309333 (671)	total: 12.2s	remaining: 3.04s
9000:	learn: 0.7364711	test: 86.7460544	best: 83.9309333 (671)	total: 13.6s	remaining: 1.51s
9999:	learn: 0.4784871	test: 86.7654536	best: 83.9309333 (671)	total

In [8]:
X_test = []

for file in files_test:
    df = pd.read_csv(test_patch + file)
    X_test.append(np.hstack((df.iloc[-1].values, 
           df.iloc[-100].values, df.iloc[-1].values - df.iloc[-100].values, df.iloc[-100:].mean(), df.iloc[-100:].std(), 
           df.iloc[-200].values, df.iloc[-1].values - df.iloc[-200].values, df.iloc[-200:].mean(), df.iloc[-200:].std(), 
           df.iloc[-300].values, df.iloc[-1].values - df.iloc[-300].values, df.iloc[-300:].mean(), df.iloc[-300:].std())))
X_test = np.array(X_test)

In [9]:
pred = model.predict(X_test)

In [10]:
df = pd.DataFrame(np.array([files_test, pred]).T, columns=['id', 'predicted'])

In [11]:
df.to_csv('task-2.csv', index=False)

In [12]:
model.save_model('model2')